In [47]:
import numpy as np
import pandas as pd
import random

In [48]:
weekday_shift_times = ["6:45-9:15am", "11:15am-1:45pm", "5:45-9:15pm"]
weekend_shift_times = ["8:45am-12:15pm", "1:45-5:15pm"]


def split_string(row):
    if type(row) == str:
        return set(row.split(", "))
    return row


In [49]:
availability = pd.read_csv("availabilities.csv")
availability['Timestamp'] = pd.to_datetime(availability['Timestamp'])
names = availability['Name'].values

for day in availability.columns[2:]:
    availability.loc[:, day] = availability[day].apply(lambda row: split_string(row))
    availability.loc[:, day+"_count"] = availability[day].apply(lambda row: len(row) if type(row) == set else 0)

    
availability


,Timestamp,Name,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday_count,Tuesday_count,Wednesday_count,Thursday_count,Friday_count,Saturday_count,Sunday_count
0,2022-05-26 23:10:00,Test,"{5:45-9:15pm, 11:15am-1:45pm}",NaN,NaN,NaN,NaN,{8:45am-12:15pm},NaN,2,0,0,0,0,1,0
1,NaT,John,{6:45-9:15am},{6:45-9:15am},{6:45-9:15am},{6:45-9:15am},{6:45-9:15am},{8:45am-12:15pm},"{1:45-5:15pm, 8:45am-12:15pm}",1,1,1,1,1,1,2
2,NaT,Harold,{11:15am-1:45pm},{11:15am-1:45pm},{11:15am-1:45pm},{11:15am-1:45pm},{11:15am-1:45pm},{1:45-5:15pm},NaN,1,1,1,1,1,1,0
3,NaT,Susie,{5:45-9:15pm},{5:45-9:15pm},{5:45-9:15pm},NaN,NaN,NaN,NaN,1,1,1,0,0,0,0
4,NaT,Brittany,{6:45-9:15am},{6:45-9:15am},{11:15am-1:45pm},{11:15am-1:45pm},{11:15am-1:45pm},NaN,NaN,1,1,1,1,1,0,0


In [50]:

schedule = pd.DataFrame([], index=weekday_shift_times+weekend_shift_times, columns=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
schedule

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
6:45-9:15am,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11:15am-1:45pm,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5:45-9:15pm,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8:45am-12:15pm,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1:45-5:15pm,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
i = 0
for time in schedule.index:
    for day in schedule.columns:
        if time in weekday_shift_times and (day == "Saturday" or day == "Sunday"):
            continue
        elif time in weekend_shift_times and day != "Saturday" and day != "Sunday":
            continue
        print(time, day)
        availability.sort_values(day + "_count", inplace=True)
        for _ in range(1):
            guards = set()
            names = availability["Name"].values
            guard_name = names[i]
            # display(availability)
            # print(guard_name)
            # print(availability.loc[availability['Name'] == guard_name, day].iloc[0])
            # print(time in availability.loc[availability['Name'] == guard_name, day].iloc[0])
            while type(
                availability.loc[availability["Name"] == guard_name, day].iloc[0]
            ) != set or (
                time
                not in availability.loc[availability["Name"] == guard_name, day].iloc[0]
                and guard_name not in guards
            ):
                i += 1
                if i == len(names):
                    i = 0
                guard_name = names[i]
            guards.add(guard_name)
            # print(guard_name)
        schedule.loc[time, day] = guards
    display(schedule)


schedule


6:45-9:15am Monday
6:45-9:15am Tuesday
6:45-9:15am Wednesday
6:45-9:15am Thursday
6:45-9:15am Friday


,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
6:45-9:15am,{Brittany},{Brittany},{John},{John},{John},NaN,NaN
11:15am-1:45pm,{Test},{Harold},{Harold},{Harold},{Harold},NaN,NaN
5:45-9:15pm,{Test},{Susie},{Susie},{Test},{Test},NaN,NaN
8:45am-12:15pm,NaN,NaN,NaN,NaN,NaN,{Susie},{Susie}
1:45-5:15pm,NaN,NaN,NaN,NaN,NaN,{Susie},{Susie}


11:15am-1:45pm Monday
11:15am-1:45pm Tuesday
11:15am-1:45pm Wednesday
11:15am-1:45pm Thursday
11:15am-1:45pm Friday


,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
6:45-9:15am,{Brittany},{Brittany},{John},{John},{John},NaN,NaN
11:15am-1:45pm,{Test},{Harold},{Harold},{Harold},{Harold},NaN,NaN
5:45-9:15pm,{Test},{Susie},{Susie},{Test},{Test},NaN,NaN
8:45am-12:15pm,NaN,NaN,NaN,NaN,NaN,{Susie},{Susie}
1:45-5:15pm,NaN,NaN,NaN,NaN,NaN,{Susie},{Susie}


5:45-9:15pm Monday
5:45-9:15pm Tuesday
5:45-9:15pm Wednesday
5:45-9:15pm Thursday


KeyboardInterrupt: 